In [1]:
import numpy as np
import pandas as pd
import datetime
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

import matplotlib.pyplot as plt
%matplotlib inline

import sys, os, gc, types
import time
from subprocess import check_output

In [2]:
root_paths = [
    "/data/kaggle-wikipedia/data2/",
    "/Users/jiayou/Dropbox/JuanCode/Kaggle/Wikipedia/data2/",
    "/Users/jiayou/Dropbox/Documents/JuanCode/Kaggle/Wikipedia/data2/"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
print(check_output(["ls", root]).decode("utf8"))

date_df.csv
key_1.csv
median_49.csv
median_weekday.csv
median_weekend.csv
page_df.csv
page_ohe.csv
sample_submission_1.csv
spider.txt
train_1.csv



In [3]:
train = pd.read_csv(root + 'train_2.csv')
train.fillna(0, inplace = True)

In [4]:
down_sample = None
if down_sample is not None:
    train = train[train.index % down_sample == 0]

In [5]:
class Heap:
    def __init__(self, isMinHeap):
        self.isMinHeap = isMinHeap
        self.data = []
        self.location = {}

    def isInOrder(self, a, b):
        if self.isMinHeap:
            return a <= b
        else:
            return a >= b

    def add(self, id, value):
        self.data.append((id, value))
        self.location[id] = self.getSize() - 1
        currentNode = self.data[-1]
        self.siftUp(currentNode)

    def remove(self, id):
        last = self.data[-1]
        if last[0] == id:
            self.data.pop()
            return
        i = self.location[id]
        del self.location[id]
        last = self.data.pop()
        self.data[i] = last
        self.location[last[0]] = i
        self.siftUp(last)
        self.siftDown(last)

    def getRoot(self):
        if len(self.data) == 0:
            return (-1, -sys.maxint)
        return self.data[0]

    def popRoot(self):
        if len(self.data) == 0:
            return None
        first = self.data[0]
        del self.location[first[0]]
        last = self.data.pop()
        if len(self.data) > 0:
            self.data[0] = last
            self.location[last[0]] = 0
            self.siftDown(last)
        return first


    def getSize(self):
        return len(self.data)

    def siftUp(self, x):
        id = x[0]
        value = x[1]
        i = self.location[x[0]]
        while i >=1 and self.isInOrder(self.data[i][1], self.data[int((i - 1)/2)][1]):
            parent_i = int((i - 1) / 2)
            temp = self.data[parent_i]
            self.data[parent_i] = self.data[i]
            self.data[i] = temp
            self.location[self.data[i][0]] = i
            self.location[self.data[parent_i][0]] = parent_i
            i = parent_i

    def siftDown(self, x):
        id = x[0]
        value = x[1]
        i = self.location[id]
        while (2*i + 1) < self.getSize():
            child_i = 2*i + 1
            child_i_2 = 2*i + 2
            if child_i_2 < self.getSize() and self.isInOrder(self.data[child_i_2][1], self.data[child_i][1]) :
                child_i = child_i_2
            if self.isInOrder(self.data[child_i][1], self.data[i][1]):
                temp = self.data[child_i]
                self.data[child_i] = self.data[i]
                self.data[i] = temp
                self.location[self.data[i][0]] = i
                self.location[self.data[child_i][0]] = child_i
            i = child_i

In [6]:
def medianSlidingWindow(nums, k, mask):
    """
        mask - 550 bool array
    """
    if nums is None or len(nums) == 0:
        return []
    n = len(nums)
    result = []
    maxHeap = Heap(False) #half smaller numbers 
    minHeap = Heap(True)

    for i in range(0, n):
        if maxHeap.getSize() == 0:
            result.append(0)
        else:
            result.append(maxHeap.getRoot()[1])
        if mask[i]:
            first = i - k
            if first >= 0 and first in maxHeap.location:
                maxHeap.remove(first)
            elif first >= 0 and first in minHeap.location:
                minHeap.remove(first)
            if maxHeap.getSize() == 0 or nums[i] <= maxHeap.getRoot()[1]:
                maxHeap.add(i, nums[i])
            else:
                minHeap.add(i, nums[i])
            if maxHeap.getSize() - minHeap.getSize() >= 2:
                top = maxHeap.popRoot()
                minHeap.add(top[0], top[1])
            elif maxHeap.getSize() - minHeap.getSize() <= -1:
                top = minHeap.popRoot()
                maxHeap.add(top[0], top[1])
    return result

In [7]:
median_df = train.copy()
train_np = train.iloc[:, 1:].values
median_np = median_df.iloc[:, 1:].values

In [15]:
def calculate_median(mask, mask_name, window):
    for i in range(len(median_df)):
    # for i in range(1000):
        nums = train_np[i, :]
        cur_median = medianSlidingWindow(nums, window, mask)
        median_np[i, :] = cur_median
        if i % 1000 == 0:
            print('{}: row {} is done'.format(mask_name, i))
    median_df.iloc[:, 1:] = median_np
    median_df.to_pickle(root+'median_{}.pkl'.format(mask_name))                          

In [12]:
date_df = pd.read_csv(root + 'date_df.csv')

params = []
params.append(([True]*len(date_df), '49_fix', 49))
params.append((date_df.isweekday.values == True, 'weekday_fix', 49))
params.append((date_df.isweekday.values == False, 'weekend_fix', 49))
params.append(([True]*len(date_df), '28', 28))
params.append((date_df.isweekday.values == True, 'weekday_28', 28))
params.append((date_df.isweekday.values == False, 'weekend_28', 28))

for i in range(7):
    params.append((date_df.dayofweek == i, 'dow{}'.format(i), 49))

In [ ]:
from multiprocessing import Process

jobs = []
for i in range(len(params)):
    p = Process(target=calculate_median, args=params[i])
    p.start()
    jobs.append(p)
    
for p in jobs:
    p.join()

print("\n\nAll done.")

In [ ]:
median_data = []
median_name = [
    '49_fix', 'weekday_fix', 'weekend_fix', 
    'dow0', 'dow1', 'dow2', 'dow3', 'dow4', 'dow5', 'dow6',
    '28', 'weekday_28', 'weekend_28'
]
for mname in median_name:
    median_data.append(pd.read_pickle(root + 'median_{}.pkl'.format(mname)))

val_days = 62
if val_days != 0:
    for j in range(len(median_name)):
        last_day = median_data[j].iloc[:, -val_days-1].values.reshape((len(median_data[j]),1))
        for i in range(-val_days, 0):
            median_data[j].iloc[:, i] = last_day

median_df = []

for i in range(len(median_name)):
    cur_median = median_data[i].melt(
        id_vars=['Page'], 
        var_name='date', 
        value_name='median_{}'.format(median_name[i])
    )
    if i != 0:
        cur_median.drop(['Page', 'date'], axis=1, inplace=True)
    median_df.append(cur_median)
    
pd.concat(median_df, axis=1).to_pickle(
    os.path.join(root, 'melted_median_62.pkl'))